# Tutorial 9: Custom Controllers

This tutorial walks through the process of defining controllers for the lateral and longitudinal movement of human-driven vehicles within a network. Such controllers may be necessary in order to model types of human behavior not already supported in SUMO. Controllers can be defined by adding to the existing controllers defined in the directory `flow/controllers/`. \
本教程介绍了在网络中定义人车横向和纵向运动控制器的过程。为了对SUMO中尚未支持的人类行为类型建模，这样的控制器可能是必要的。可以通过添加到“flow/ Controllers /”目录中定义的现有控制器来定义控制器。

Here, we will discuss Flow's `BaseController` class and then build two controllers: a longitudinal Intelligent Driver Model controller [CITE] and a lateral controller that attempts to move all vehicles into the same lane.
在这里，我们将讨论Flow的‘base econtroller’类，然后构建两个控制器:一个纵向智能驾驶员模型控制器[CITE]和一个横向控制器，试图将所有车辆移动到同一车道。

When adding a custom controller, ensure changes are reflected in `flow/controllers/__init__.py` under the import statements as well as in the list `__all__`. 
在添加自定义控制器时，确保更改反映在“flow/controllers/ init__”中。在import语句下，以及在列表“_all__”中。

## 1 Longitudinal Controller 纵向控制器

### 1.1 BaseController

Flow's `BaseController` class is an abstract class to use when implementing longitudinal controllers. It includes failsafe methods and the `get_action` method called by Flow's `core.base` module. `get_action` adds noise to actions and runs failsafes, if specified. `BaseController` does not implement `get_accel`; that method should be implemented in any controllers that are subclasses of `BaseController`. 

Flow的‘BaseController’类是在实现纵向控制器时使用的抽象类。它包括故障安全方法和由Flow' core '调用的' get_action '方法。基本的模块。' get_action '为操作添加噪音，并运行故障保险，如果指定的话。' BaseController '没有实现' get_accel ';该方法应该在任何属于‘BaseController’子类的控制器中实现。

As such, any longitudinal controller must import `BaseController`. We also import NumPy in order to use some mathematical functions.
因此，任何纵向控制器都必须导入‘BaseController’。为了使用一些数学函数，我们还导入了NumPy。

In [ ]:
import numpy as np

from flow.controllers.base_controller import BaseController

### 1.2 Controller Initialization 控制器初始化

Here we initialize an IDM controller class and the `__init__` function storing class attributes.
在这里，我们初始化了一个IDM控制器类和存储类属性的“_init__”函数。

The Intelligent Driver Model is a car-following model specifying vehicle dynamics by a differential equation for acceleration $\dot{v}$. The differential equation follows:
智能驾驶员模型是一种用加速度微分方程来描述车辆动力学的车辆跟踪模型。微分方程如下:

$$\dot{v} = a \left[ 1- \left( \frac{v}{v_0} \right)^\delta -\left( \frac{s^*}{h} \right)^2 \right] \textbf{, with } \ s^* := s_0 + \max \left( 0, vT + \frac{v\Delta v}{2\sqrt{ab}} \right)$$

The IDM parameters are: desired speed $v_0$, time gap $T$, min gap $s_0$, acceleration exponent $\delta$, acceleration term $a$, and comfortable deceleration $b$. $h$ is the vehicle headway (the distance to the vehicle ahead) and $\Delta v$ is the velocity difference compared to the lead vehicle (current velocity - lead velocity).
IDM参数:理想转速𝑣0,时间差距𝑇,𝑠0最小差距,加速指数𝛿,减速𝑏加速度项𝑎,舒适。ℎ是车辆进展(距离前方的车辆)和Δ𝑣速度差异而导致车辆(当前速度,导致速度)。

In [ ]:
class IDMController(BaseController):
    def __init__(self, veh_id, v0=30, T=1, a=1, b=1.5, 
                 delta=4, s0=2, s1=0, time_delay=0.0, 
                 dt=0.1, noise=0, fail_safe=None, car_following_params=None):
        """
        veh_id: str
            unique vehicle identifier
        car_following_params: SumoCarFollowingParams 
            see parent class
        v0: float, optional
            desirable velocity理想速度, in m/s (default: 30)
        T: float, optional
            safe time headway安全时间进展, in s (default: 1)
        b: float, optional
            comfortable deceleration舒适的减速, in m/s2 (default: 1.5)
        delta: float, optional
            acceleration exponent加速指数 (default: 4)
        s0: float, optional
            linear jam distance线性干扰的距离, in m (default: 2)
        s1: float, optional
            nonlinear jam distance非线性干扰的距离, in m (default: 0)
        dt: float, optional
            timestep, in s (default: 0.1)
        noise: float, optional
            std dev of normal perturbation to the acceleration
            标准摄动对加速度的std发展 (default: 0)
        fail_safe: str, optional
            type of flow-imposed failsafe the vehicle should posses, defaults
            to no failsafe 车辆应该拥有的、默认的、流强制的故障安全类型
            默认没有故障保险(没有)(None)
        """
        
        BaseController.__init__(self, veh_id, car_following_params,
                                delay=time_delay, fail_safe=fail_safe,
                                noise=noise)
        self.v0 = v0
        self.T = T
        self.a = a
        self.b = b
        self.delta = delta
        self.s0 = s0
        self.s1 = s1
        self.dt = dt

### 1.3 Acceleration Command 加速命令
 
Next, we implement the acceleration equation specified by IDM: 接下来，我们实现IDM指定的加速度方程:

$$\dot{v} = a \left[ 1- \left( \frac{v}{v_0} \right)^\delta -\left( \frac{s^*}{h} \right)^2 \right] \textbf{, with } \ s^* := s_0 + \max \left( 0, vT + \frac{v\Delta v}{2\sqrt{ab}} \right)$$

The vehicle's velocity `v` is fetched by getter method `get_speed` of the environment's vehicles object, as is the id of the lead vehicle `lead_id` and headway `h`. 

Input-checking to ensure that overly small headways are not used is performed, as well as a step to set $s^*$ properly when no car is ahead of the vehicle being controlled. If there is a lead vehicle, $s^*$ is calculated as described, and then the IDM acceleration is returned.
车辆的速度v通过获取环境车辆对象的get_speed方法获取，同样获取的还有前车的id lead_id和车头时距h。


输入检查以确保过于小的执行进展不习惯,以及一套一步𝑠∗正确当没有车的车辆被控制。如果有辆车,𝑠∗计算描述,然后返回IDM加速度。

In [ ]:
class IDMController(BaseController):
    def __init__(self, veh_id, v0=30, T=1, a=1, b=1.5, 
                 delta=4, s0=2, s1=0, time_delay=0.0, 
                 dt=0.1, noise=0, fail_safe=None, car_following_params=None):
        """Docstring eliminated here for brevity"""
        BaseController.__init__(self, veh_id, car_following_params,
                                delay=time_delay, fail_safe=fail_safe,
                                noise=noise)
        self.v0 = v0
        self.T = T
        self.a = a
        self.b = b
        self.delta = delta
        self.s0 = s0
        self.s1 = s1
        self.dt = dt

        
    ##### Below this is new code #####
    def get_accel(self, env):
        v = env.k.vehicle.get_speed(self.veh_id)
        lead_id = env.k.vehicle.get_leader(self.veh_id)
        h = env.k.vehicle.get_headway(self.veh_id)

        # negative headways may be registered by sumo at intersections/
        # junctions. Setting them to 0 causes vehicles to not move; therefore,
        # we maintain these negative headways to let sumo control the dynamics
        # as it sees fit at these points.
        if abs(h) < 1e-3:
            h = 1e-3

        if lead_id is None or lead_id == '':  # no car ahead
            s_star = 0
        else:
            lead_vel = env.k.vehicle.get_speed(lead_id)
            s_star = self.s0 + max(
                0,
                v * self.T + v*(v-lead_vel) / (2*np.sqrt(self.a*self.b)))

        return self.a * (1 - (v/self.v0)**self.delta - (s_star/h)**2)

## 2 Lateral Controller 横向控制器

### 2.1 BaseLaneChangeController

In this section we will implement a lane-change controller that sends lane-change commands to move a vehicle into lane 2. Flow includes a BaseLaneChangeController abstract class that functions similarly to the BaseController class, implementing safety-checking utility methods for control.
在本节中，我们将实现一个变道控制器，它发送变道命令来将车辆移到第二车道。Flow包含一个BaseLaneChangeController抽象类，它的功能类似于BaseController类，实现了用于控制的安全检查实用方法。

First, we import the BaseLaneChangeController object and define a lane-change controller class, but leave method definition until the next step. 
首先，我们导入BaseLaneChangeController对象并定义一个lane-change控制器类，但是将方法定义留到下一步。

In [ ]:
from flow.controllers.base_lane_changing_controller import BaseLaneChangeController

class LaneZeroController(BaseLaneChangeController):
    """A lane-changing model used to move vehicles into lane 0."""
    pass

### 2.2 Lane-Change Command

Lane-change controllers must implement the method `get_lane_change_action`. Actions in Flow are specified as directions, which are a number out of `[-1, 0, 1]`. Lane 0 is the farthest-right, so the direction -1 is a lane change to the right. 

lane_change控制器必须实现get_lane_change_action方法。Flow中的动作被指定为方向，方向是[- 1,0,1]中的一个数字。0号车道是最右边的，所以-1方向是右转车道。


This `get_lane_change_action` implementation fetches the current lane the vehicle is in, using the `get_lane` method of the Vehicles object and passing in `self.veh_id`. If the vehicle is in a lane different from lane 0, it must have a lane number above 0, since lane numbers are positive in SUMO. In that case, a lane-change to the right is specified by returning the direction -1. If the vehicle is in lane 0, then the direction 0 is returned.

这个get_lane_change_action实现使用Vehicles对象的get_lane方法获取车辆所在的当前车道，并传入self.veh_id。如果车辆在不同于0道的车道上，它的车道号必须大于0，因为在相扑比赛中，车道号是正的。在这种情况下，通过返回方向-1指定向右的车道更改。如果车辆在0车道，则返回方向0。

In [ ]:
class LaneZeroController(BaseLaneChangeController):
    """A lane-changing model used to move vehicles into lane 0."""

    ##### Below this is new code #####
    def get_lane_change_action(self, env):
        current_lane = env.k.vehicle.get_lane(self.veh_id)
        if current_lane > 0:
            return -1
        else:
            return 0